In [1]:
!pip install langchain faiss-gpu jq
!pip install accelerate
!pip install transformers
!pip install -U langchain-community
!pip install tiktoken
!pip install sentence-transformers

In [2]:
!pip install tensorflow[and-cuda]
!pip install torch~=2.3.0 torch_xla[tpu]~=2.3.0 -f https://storage.googleapis.com/libtpu-releases/index.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 845.8/845.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.1/720.1 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 56.1 MB/s eta 0:00:00
  Preparing metadata (

In [3]:
#Do not run this cell if not using Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from langchain import HuggingFacePipeline
from accelerate import Accelerator
from langchain.text_splitter import TokenTextSplitter,CharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

In [5]:
import os
from huggingface_hub import notebook_login
os.environ['HUGGINGFACEHUB_API_TOKEN']='YOUR_HF_API_TOKEN'
notebook_login()

In [6]:
from langchain.document_loaders import JSONLoader

# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["year"] = record.get("pub_date").get('year')
    metadata["month"] = record.get("pub_date").get('month')
    metadata["day"] = record.get("pub_date").get('day')
    metadata["title"] = record.get("article_title")

    return metadata

#Set file path to wherever JSON files from pubmed_download.py are located
loader1 = JSONLoader(
    file_path='/content/drive/MyDrive/pubmed_background.json',
    jq_schema='.[]',
    content_key='article_abstract',
    metadata_func=metadata_func)

loader2 = JSONLoader(
    file_path='/content/drive/MyDrive/pubmed_reference.json',
    jq_schema='.[]',
    content_key='article_abstract',
    metadata_func=metadata_func)
data1 = loader1.load()
data2 = loader2.load()

In [7]:
print(f"{len(data1)} pubmed articles are loaded!")
print(f"{len(data2)} pubmed articles are loaded!")

730 pubmed articles are loaded!
8313 pubmed articles are loaded!


In [8]:
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=50)
chunks1 = text_splitter.split_documents(data1)
print(f"{len(chunks1)} chunks are created!")
chunks2 = text_splitter.split_documents(data2)
print(f"{len(chunks2)} chunks are created!")

1584 chunks are created!
16443 chunks are created!


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
modelPath = "intfloat/e5-large-unsupervised"
embeddings = HuggingFaceEmbeddings(
  model_name = modelPath,
  model_kwargs = {'device':'cpu'},
  encode_kwargs={'normalize_embeddings':False})

# Using faiss index
from langchain.vectorstores import FAISS
db1 = FAISS.from_documents(chunks1, embeddings) #db1 = background folder


db2 = FAISS.from_documents(chunks2, embeddings) #db2 = reference folder

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [11]:

import torch


model_id = "meta-llama/Meta-Llama-3-8B" #Set model based on huggingface directory

tokenizer = AutoTokenizer.from_pretrained(model_id,offload_buffers=True)
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=False, device_map='auto')

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1048)
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0.5}
)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [12]:
# Define the langchain pipeline

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import time

PROMPT_TEMPLATE = """You are a medical research assistant to help write an academic research paper.
    Construct a five paragraph essay Outline about the Topic using the provided Context only.
    Your outline should be in an academic tone.
    Context: {context}
    Topic: {question}
    Outline:
"""
PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

#For background info, define retriever as background info and set task to develop topic sentences and a paper outline
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db1.as_retriever(k=2),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

In [13]:
PROMPT_TEMPLATE = """You are a medical research assistant to help write an academic research paper.
    Answer the Question using the provided Context only.
    Your answer should be based on the Outline provided, written in an academic tone and consist of at least 5 paragraphs.
    Context: {context}
    Outline: {question}
    Answer:
"""
PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

#For reference info, define retriever as reference info and set task to receive outline and produce an essay of at least 5 paragraphs
qa_chain2 = RetrievalQA.from_chain_type(
    llm,
    retriever=db2.as_retriever(k=2),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

In [16]:
#Make outline based on a topic sentence
start_time = time.time()
#Query
query = "The topic is COVID-19 and the symptoms associated with it."
result = qa_chain({"query": query})
#Execution Time
print(f"\n--- {time.time() - start_time} seconds ---")
print(result['result'].strip())
#Metadata
titles = ['\t-'+doc.metadata['title'] for doc in result['source_documents']]
print("\n\nThe provided answer is based on the following PubMed articles:\t")
print("\n".join(set(titles)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- 434.6669805049896 seconds ---
You are a medical research assistant to help write an academic research paper.
    Construct a five paragraph essay Outline about the Topic using the provided Context only.
    Your outline should be in an academic tone.
    Context:  investigate and quantify the negative impact that the COVID-19 pandemic has industry-sponsored clinical trials, both in the USA and worldwide. We find a negative correlation between the severity of the COVID-19 pandemic and clinical trial screening rate, with the relationship being strongest during the first three months of the pandemic compared to the entire duration of the pandemic. This negative statistical relationship holds across therapeutic areas, across states in the USA despite the heterogeneity of responses at the state-level, and across countries. This work has significant implications for the management of clinical trials worldwide in response to the fluctuating severity of COVID-19 moving

 highlighted that 

In [18]:
#copy and paste the outline generated by db1 into the qa chain for db2 to write the essay.
start_time = time.time()
#Query
query = """Outline:
        1. Introduction: Introduce the topic of COVID-19 and its symptoms.
        2. Symptoms: Discuss the different symptoms associated with COVID-19.
        3. Diagnosis: Describe how COVID-19 is diagnosed.
        4. Treatment: Discuss the treatment options for COVID-19.
        5. Prevention: Provide tips on how to prevent COVID-19.
        6. Conclusion: Summarize the main points of the article and provide a call to action."""
result = qa_chain2({"query": query})
#Execution Time
print(f"\n--- {time.time() - start_time} seconds ---")
print(result['result'].strip())
#Metadata
titles = ['\t-'+doc.metadata['title'] for doc in result['source_documents']]
print("\n\nThe provided answer is based on the following PubMed articles:\t")
print("\n".join(set(titles)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- 192.03601717948914 seconds ---
You are a medical research assistant to help write an academic research paper.
    Answer the Question using the provided Context only.
    Your answer should be based on the Outline provided, written in an academic tone and consist of at least 5 paragraphs.
    Context: Patients with a history of COVID-19 infection may suffer from different physical problems. This study aimed to investigate the effect of corrective and breathing exercises on improving respiratory function among patients with a history of COVID-19 infection.

SARS-CoV-2 infections caused mild-to-moderate illness. However, a sizable portion of infected people experience a rapid progression of hyper-inflammatory and hypoxic respiratory illness that necessitates an effective and safer remedy to combat COVID-19.

As the third year of the SARS-CoV-2 pandemic approaches, COVID-19 continues to cause substantial morbidity and mortality due to waning vaccine efficacy and the emergence of new,